# Retrieval based Chatbots

In [1]:
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle

import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD
import random

words=[]
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open('C:\\Reva\\BA06\\2nd Year\\capstone_project\\chatbot\\chatbot-python-project-data-codes\\creditcard_intents.json').read()
intents = json.loads(data_file)

In [2]:
for intent in intents['intents']:
    for pattern in intent['patterns']:

        #tokenize each word
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        #add documents in the corpus
        documents.append((w, intent['tag']))

        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [3]:
# lemmatize, lower each word and remove duplicates
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))
# sort classes
classes = sorted(list(set(classes)))
# documents = combination between patterns and intents
print (len(documents), "documents")
# classes = intents
print (len(classes), "classes", classes)
# words = all words, vocabulary
print (len(words), "unique lemmatized words", words)

pickle.dump(words,open('C:\\Reva\\BA06\\2nd Year\\capstone_project\\chatbot\\chatbot-python-project-data-codes\\words.pkl','wb'))
pickle.dump(classes,open('C:\\Reva\\BA06\\2nd Year\\capstone_project\\chatbot\\chatbot-python-project-data-codes\\classes.pkl','wb'))

31 documents
12 classes ['card', 'category', 'ccategory', 'fcategory', 'goodbye', 'greeting', 'lcategory', 'noanswer', 'rcategory', 'scategory', 'tcategory', 'thanks']
46 unique lemmatized words ["'s", ',', '10000', '20000', '30000', '40000', 'anyone', 'are', 'awesome', 'bye', 'card', 'cashback', 'chatting', 'credit', 'day', 'for', 'fuel', 'good', 'goodbye', 'hello', 'helpful', 'helping', 'hey', 'hi', 'hola', 'how', 'i', 'is', 'later', 'lifestyle', 'me', 'next', 'nice', 'reward', 'see', 'shopping', 'thank', 'thanks', 'that', 'there', 'till', 'time', 'to', 'travel', 'want', 'you']


In [4]:
# create our training data
training = []
# create an empty array for our output
output_empty = [0] * len(classes)
# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)
# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")

Training data created


C:\Users\madhu\AppData\Local\Temp/ipykernel_25080/2748295590.py:24: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  training = np.array(training)


In [5]:
# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

#fitting and saving the model 
hist = model.fit(np.array(train_x), np.array(train_y), epochs=100, batch_size=5, verbose=1)
model.save('C:\\Reva\\BA06\\2nd Year\\capstone_project\\chatbot\\chatbot-python-project-data-codes\\chatbot_model.h5', hist)

print("model created")

C:\ProgramData\Anaconda3\lib\site-packages\keras\optimizers\optimizer_v2\gradient_descent.py:108: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


Epoch 1/100
7/7 [==============================] - 1s 3ms/step - loss: 2.4833 - accuracy: 0.0968
Epoch 2/100
7/7 [==============================] - 0s 2ms/step - loss: 2.4154 - accuracy: 0.0645
Epoch 3/100
7/7 [==============================] - 0s 2ms/step - loss: 2.3311 - accuracy: 0.0645
Epoch 4/100
7/7 [==============================] - 0s 2ms/step - loss: 2.2412 - accuracy: 0.2581
Epoch 5/100
7/7 [==============================] - 0s 3ms/step - loss: 2.1396 - accuracy: 0.2903
Epoch 6/100
7/7 [==============================] - 0s 3ms/step - loss: 2.0974 - accuracy: 0.2581
Epoch 7/100
7/7 [==============================] - 0s 3ms/step - loss: 2.0380 - accuracy: 0.1935
Epoch 8/100
7/7 [==============================] - 0s 2ms/step - loss: 1.9532 - accuracy: 0.3548
Epoch 9/100
7/7 [==============================] - 0s 3ms/step - loss: 2.1259 - accuracy: 0.2581
Epoch 10/100
7/7 [==============================] - 0s 3ms/step - loss: 1.8595 - accuracy: 0.3548
Epoch 11/100
7/7 [===========

In [6]:
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import pickle
import numpy as np

from keras.models import load_model
model = load_model('C:\\Reva\\BA06\\2nd Year\\capstone_project\\chatbot\\chatbot-python-project-data-codes\\chatbot_model.h5')
import json
import random
intents = json.loads(open('C:\\Reva\\BA06\\2nd Year\\capstone_project\\chatbot\\chatbot-python-project-data-codes\\creditcard_intents.json').read())
words = pickle.load(open('C:\\Reva\\BA06\\2nd Year\\capstone_project\\chatbot\\chatbot-python-project-data-codes\\words.pkl','rb'))
classes = pickle.load(open('C:\\Reva\\BA06\\2nd Year\\capstone_project\\chatbot\\chatbot-python-project-data-codes\\classes.pkl','rb'))

In [7]:
def clean_up_sentence(sentence):
    # tokenize the pattern - split words into array
    sentence_words = nltk.word_tokenize(sentence)
    # stem each word - create short form for word
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words
# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence

def bow(sentence, words, show_details=True):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words - matrix of N words, vocabulary matrix
    bag = [0]*len(words) 
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                # assign 1 if current word is in the vocabulary position
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)
    return(np.array(bag))

def predict_class(sentence, model):
    # filter out predictions below a threshold
    p = bow(sentence, words,show_details=False)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list

In [8]:
def getResponse(ints, intents_json):
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if(i['tag']== tag):
            result = random.choice(i['responses'])
            break
    return result

def chatbot_response(text):
    ints = predict_class(text, model)
    res = getResponse(ints, intents)
    return res

In [9]:
#Creating GUI with tkinter
import tkinter
from tkinter import *


def send():
    msg = EntryBox.get("1.0",'end-1c').strip()
    EntryBox.delete("0.0",END)

    if msg != '':
        ChatLog.config(state=NORMAL)
        ChatLog.insert(END, "You: " + msg + '\n\n')
        ChatLog.config(foreground="#442265", font=("Calibri", 10 ))

        res = chatbot_response(msg)
        ChatLog.insert(END, "Bot: " + res + '\n\n')

        ChatLog.config(state=DISABLED)
        ChatLog.yview(END)

base = Tk()
base.title("Hello")
base.geometry("400x500")
base.resizable(width=FALSE, height=FALSE)

#Create Chat window
ChatLog = Text(base, bd=0, bg="white", height="8", width="50", font="Calibri",)

ChatLog.config(state=DISABLED)

#Bind scrollbar to Chat window
scrollbar = Scrollbar(base, command=ChatLog.yview, cursor="heart")
ChatLog['yscrollcommand'] = scrollbar.set

#Create Button to send message
SendButton = Button(base, font=("Calibri",10,'bold'), text="Send", width="12", height=5,
                    bd=0, bg="#32de97", activebackground="#3c9d9b",fg='#ffffff',
                    command= send )

#Create the box to enter message
EntryBox = Text(base, bd=0, bg="white",width="29", height="5", font="Calibri")
#EntryBox.bind("<Return>", send)


#Place all components on the screen
scrollbar.place(x=376,y=6, height=386)
ChatLog.place(x=6,y=6, height=386, width=370)
EntryBox.place(x=128, y=401, height=90, width=265)
SendButton.place(x=6, y=401, height=90)

base.mainloop()

1/1 [==============================] - 0s 15ms/step
